In [1]:
# Constants
DATA_PATH   = "../Data/Processed/3_engineered_df.pkl"

TECH_JOBS = ['Techjobs']

CORE_COLS = ['VersionControlSystem',
             'Languages',
             'Databases',
             'Platforms',
             'WebFrameworks',
             'MiscTech',
             'ToolsTech',
             'CollabTools'
]


MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "tech_jobs_predictions"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
from pathlib import Path
import os

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

_____

### Functions

_____

In [3]:
# Load dataset and make a copy
eng_df = pd.read_pickle(DATA_PATH)
df = eng_df.copy()

In [4]:
df

Techjobs                 \
      Data scientist or machine learning specialist Engineer, data   
2                                                 1              1   
3                                                 0              0   
9                                                 0              0   
10                                                0              0   
12                                                0              0   
...                                             ...            ...   
73262                                             0              0   
73263                                             0              0   
73264                                             1              0   
73265                                             0              0   
73266                                             0              0   

                                                                           \
      Data or business analyst Developer, back-end Database administrator   
2                            0                   0                      0   
3                            0                   0                      0   
9                            0                   0                      0   
10                           0                   1                      0   
12                           0                   0                      0   
...                        ...                 ...                    ...   
73262                        0                   0                      0   
73263                        0                   1                      0   
73264                        0                   0                      0   
73265                        0                   0                      0   
73266                        0                   0                      0   

                                                                             \
      Developer, mobile Developer, full-stack Cloud infrastructure engineer   
2                     0                     0                             0   
3                     0                     1                             0   
9                     1                     0                             0   
10                    0                     1                             0   
12                    0                     1                             0   
...                 ...                   ...                           ...   
73262                 0                     0                             0   
73263                 0                     0                             0   
73264                 0                     0                             0   
73265                 0                     1                             0   
73266                 0                     0                             0   

                                                                         \
      Developer, embedded applications or devices Developer, QA or test   
2                                               0                     0   
3                                               0                     0   
9                                               0                     0   
10                                              0                     0   
12                                              0                     0   
...                                           ...                   ...   
73262                                           0                     0   
73263                                           0                     0   
73264                                           0                     0   
73265                                           0                     0   
73266                                           0                     0   

                                                            \
      System administrator Scientist Security professional   
2 

### Deal with Imbalanced Dataset

In [6]:
jobs_df =df['Techjobs'].copy()
jobs_df.sum(axis=0).sort_values()

Blockchain                                         736
Security professional                              801
Developer, game or graphics                       1009
Scientist                                         1122
Developer, QA or test                             1425
Data or business analyst                          1666
Database administrator                            1755
Academic researcher                               1819
Engineer, data                                    2091
System administrator                              2099
Developer, embedded applications or devices       2185
Data scientist or machine learning specialist     2268
Cloud infrastructure engineer                     2813
DevOps specialist                                 3350
Developer, mobile                                 4613
Developer, desktop or enterprise applications     5734
Developer, front-end                             10172
Developer, back-end                              19001
Developer,

In [7]:
len(jobs_df)

46408

In [23]:
# DownSample majority classes and OverSample minority Classes
samples_per_class = 1400
resampled_jobs = []

for job in jobs_df.columns:
    sub_df = jobs_df.loc[jobs_df[job] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=42)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=42) 
    
    resampled_jobs.append(sub_df)

In [24]:
jobs_df  = pd.concat(resampled_jobs)
jobs_df.sum(axis=0).sort_values()

Blockchain                                       3695
Developer, game or graphics                      3858
Developer, QA or test                            3989
Security professional                            4146
Developer, embedded applications or devices      4355
Scientist                                        4764
Developer, mobile                                4781
Data or business analyst                         4824
Developer, front-end                             4842
Engineer, data                                   4850
Cloud infrastructure engineer                    4905
DevOps specialist                                5059
Database administrator                           5142
Data scientist or machine learning specialist    5204
Academic researcher                              5233
System administrator                             5236
Developer, desktop or enterprise applications    5579
Developer, full-stack                            8346
Developer, back-end         

In [27]:
len(jobs_df)

26600

In [30]:
balanced_df = df.loc[jobs_df.index].copy()

In [32]:
X = balanced_df.drop(balanced_df[TECH_JOBS],axis = 1).droplevel(0,axis=1).copy()
y = balanced_df[TECH_JOBS].droplevel(0,axis = 1).copy()

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    y,test_size=0.20,
                                                    random_state=42)

In [39]:
len(X_train)

21280

### Create a Test Set

### Intialize MLflow

In [11]:
# Create Directories
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok=True)
Path(LOG_PATH).mkdir(parents=True, exist_ok=True)

In [12]:
# Initialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

### Playground

In [7]:
one_job_df = eng_df[jobs_df.sum(axis=1) ==1]

In [8]:
len(one_job_df)

23154

In [9]:
one_job_df[TECH_JOBS].sum(axis =0).sort_values()

Techjobs  Blockchain                                         56
          Database administrator                             62
          Scientist                                         127
          Security professional                             146
          Cloud infrastructure engineer                     170
          Developer, game or graphics                       215
          System administrator                              215
          DevOps specialist                                 278
          Data or business analyst                          315
          Developer, QA or test                             351
          Engineer, data                                    370
          Academic researcher                               440
          Data scientist or machine learning specialist     582
          Developer, embedded applications or devices       659
          Developer, desktop or enterprise applications     869
          Developer, mobile             

In [ ]:
# apply smote to 1 job df
x_train = one_job_df.drop(one_job_df[TECH_JOBS],axis = 1).droplevel(0,axis=1).copy()
y_train = one_job_df[TECH_JOBS].droplevel(0,axis = 1).copy()

In [ ]:
#convert labels to array or series because smote doesn't accept dataframe as labels
y_train_1 = y_train[(y_train['Developer, mobile'] == 1) | (y_train['Developer, front-end'] == 1)|(y_train['Developer, back-end'] == 1)|(y_train['Developer, full-stack'] == 1)]
y_train_2 = y_train[~((y_train['Developer, mobile'] == 1) | (y_train['Developer, front-end'] == 1)|(y_train['Developer, back-end'] == 1)|(y_train['Developer, full-stack'] == 1))]

In [ ]:
y_train_1_array = y_train_1.to_numpy()
y_train_2_array = y_train_2.to_numpy()

In [ ]:
y_train_array = y_train.to_numpy()

In [ ]:
y_train

,Data scientist or machine learning specialist,"Engineer, data",Data or business analyst,"Developer, back-end",Database administrator,"Developer, mobile","Developer, full-stack",Cloud infrastructure engineer,"Developer, embedded applications or devices","Developer, QA or test",System administrator,Scientist,Security professional,"Developer, game or graphics","Developer, front-end",Blockchain,"Developer, desktop or enterprise applications",DevOps specialist,Academic researcher
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
17,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73260,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
73261,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
73262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
73263,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
over = SMOTE(random_state=42, k_neighbors=5)
under= RandomUnderSampler(random_state=42,sampling_strategy={3:1000,5:1000,6:1000,14:1000})
x_under, y_under= under.fit_resample(x_train, y_train_array)
x_over, y_over = over.fit_resample(x_under, y_under)

In [ ]:
y_train_under= pd.DataFrame(y_under, index= x_under.index,columns = y_train.columns)

In [ ]:
y_train_under.sum(axis=0).sort_values()

Blockchain                                         56
Database administrator                             62
Scientist                                         127
Security professional                             146
Cloud infrastructure engineer                     170
Developer, game or graphics                       215
System administrator                              215
DevOps specialist                                 278
Data or business analyst                          315
Developer, QA or test                             351
Engineer, data                                    370
Academic researcher                               440
Data scientist or machine learning specialist     582
Developer, embedded applications or devices       659
Developer, desktop or enterprise applications     869
Developer, full-stack                            1000
Developer, mobile                                1000
Developer, front-end                             1000
Developer, back-end         

In [ ]:
y_train_over = pd.DataFrame(y_over, index= x_over.index,columns = y_train.columns)

In [ ]:
y_train_over.sum(axis = 0)

Data scientist or machine learning specialist    1000
Engineer, data                                   1000
Data or business analyst                         1000
Developer, back-end                              1000
Database administrator                           1000
Developer, mobile                                1000
Developer, full-stack                            1000
Cloud infrastructure engineer                    1000
Developer, embedded applications or devices      1000
Developer, QA or test                            1000
System administrator                             1000
Scientist                                        1000
Security professional                            1000
Developer, game or graphics                      1000
Developer, front-end                             1000
Blockchain                                       1000
Developer, desktop or enterprise applications    1000
DevOps specialist                                1000
Academic researcher         

In [ ]:
len(y_train_over)

19000

In [33]:
from sklearn.model_selection import StratifiedKFold

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


for train_index, test_index in skf.split(X, y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.